### 1. 키워드파일 업로드

In [1]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel

# CSV 파일에서 클러스터링 결과 로드
df_inheritance = pd.read_csv(r'C:\Users\user\Documents\no-f\241030\df_inheritance_keyword.csv')

c:\Users\user\.conda\envs\nof\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2. 사건명가져오기 위해서 판례목록 데이터 불러오기

In [2]:
# 사건 리스트 로드
inheritance_list = pd.read_csv('inheritance_list.csv')

# 사건명 추가를 위해 df_inheritance 데이터프레임과 병합
df_inheritance = df_inheritance.merge(inheritance_list[['사건번호', '사건명']], on='사건번호', how='left')

### 3. 임베딩해서 csv파일로 저장하기

In [3]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel

# KOBERT 모델과 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert')

# 문장 임베딩 함수
def encode_sentence(sentence):
    if pd.notnull(sentence):
        inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).numpy()
    return None

# 클러스터별 판례 내용과 키워드 임베딩 생성
for cluster_num in range(1, 11):  # 클러스터 1부터 10까지
    cluster_cases = df_inheritance[df_inheritance['클러스터'] == cluster_num]
    
    df_inheritance.loc[df_inheritance['클러스터'] == cluster_num, '판례내용이유임베딩'] = \
        cluster_cases['판례내용_이유'].apply(encode_sentence)
    
    df_inheritance.loc[df_inheritance['클러스터'] == cluster_num, '키워드임베딩'] = \
        cluster_cases['키워드'].apply(encode_sentence)
    
    df_inheritance.loc[df_inheritance['클러스터'] == cluster_num, '판결요지임베딩'] = \
        cluster_cases['판결요지'].apply(encode_sentence)
    
    df_inheritance.loc[df_inheritance['클러스터'] == cluster_num, '판시사항임베딩'] = \
        cluster_cases['판시사항'].apply(encode_sentence)

# CSV 파일로 저장
df_inheritance.to_csv('df_inheritance_kobertmodel2.csv', index=False)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [2]:
df_inheritance.head(2)

NameError: name 'df_inheritance' is not defined

In [11]:
df_inheritance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4166 entries, 0 to 4165
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   판례정보일련번호       4166 non-null   int64 
 1   사건번호           4166 non-null   object
 2   선고             4166 non-null   object
 3   판결유형           4166 non-null   object
 4   판시사항           3250 non-null   object
 5   판결요지           2932 non-null   object
 6   참조조문           3225 non-null   object
 7   참조판례           1935 non-null   object
 8   판례내용           4166 non-null   object
 9   판례내용_상단        4166 non-null   object
 10  판례내용_이유        4166 non-null   object
 11  판례내용_이유_전처리    4146 non-null   object
 12  판례내용_이유_불용어제거  4131 non-null   object
 13  클러스터           4166 non-null   int64 
 14  키워드            4166 non-null   object
 15  사건명            3924 non-null   object
 16  판례내용이유임베딩      3873 non-null   object
 17  키워드임베딩         3873 non-null   object
 18  판결요지임베딩        2701 non-null

### 4. 유사도 계산해서 추천사건명 출력하기

#### 판결요지

In [10]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 사용자 입력
user_keyword = "토지"  # 사용자가 입력한 키워드
user_keyword_embedding = encode_sentence(user_keyword)

# 판례내용 임베딩 유사도 계산 및 사건명 추출
content_similarity_scores = []

for idx, row in df_inheritance.iterrows():
    if row['판결요지임베딩'] is not None and not np.isnan(row['판결요지임베딩']).any():
        # 판례내용 임베딩과 사용자 키워드 임베딩 간의 유사도 계산
        content_similarity = cosine_similarity(user_keyword_embedding, row['판결요지임베딩']).flatten()[0]
        content_similarity_scores.append((row['사건번호'], content_similarity))

# 유사도 기준으로 정렬 및 상위 N개 선택
sorted_content_scores = sorted(content_similarity_scores, key=lambda x: x[1], reverse=True)
top_n_contents = sorted_content_scores[:10]

# 추천 사건명 출력
print("사용자 키워드와 관련된 사건명 :")
for 사건번호, score in top_n_contents:
    사건명 = df_inheritance.loc[df_inheritance['사건번호'] == 사건번호, '사건명'].values
    사건명 = 사건명[0] if 사건명.size > 0 else "사건명 없음"
    print(f"사건번호: {사건번호}, 사건명: {사건명}, 유사도: {score:.4f}")

사용자 키워드와 관련된 사건명 :
사건번호: 4294민상168, 사건명: 덕대계약이행완료확인, 유사도: 0.5380
사건번호: 81다카550, 사건명: 소유권이전등기말소, 유사도: 0.5114
사건번호: 86다카1312, 사건명: 소유권이전등기말소등, 유사도: 0.4960
사건번호: 4294민상1255, 사건명: 가옥명도, 유사도: 0.4863
사건번호: 62다343, 사건명: 부동산소유권이전등기말소, 유사도: 0.4846
사건번호: 87다카663, 사건명: 소유권이전등기말소등, 유사도: 0.4839
사건번호: 90다카17214, 사건명: 소유권이전등기말소, 유사도: 0.4827
사건번호: 80구844, 사건명: 취득세부과처분취소, 유사도: 0.4824
사건번호: 90다카3789, 사건명: 소유권이전등기말소등, 유사도: 0.4777
사건번호: 91다7750, 사건명: 소유권이전등기말소등, 유사도: 0.4773


#### 판례내용_이유

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 사용자 입력
user_keyword = "토지"  # 사용자가 입력한 키워드
user_keyword_embedding = encode_sentence(user_keyword)

# 판례내용 임베딩 유사도 계산 및 사건명 추출
content_similarity_scores = []

for idx, row in df_inheritance.iterrows():
    if row['판례내용이유임베딩'] is not None and not np.isnan(row['판례내용이유임베딩']).any():
        # 판례내용 임베딩과 사용자 키워드 임베딩 간의 유사도 계산
        content_similarity = cosine_similarity(user_keyword_embedding, row['판례내용이유임베딩']).flatten()[0]
        content_similarity_scores.append((row['사건번호'], content_similarity))

# 유사도 기준으로 정렬 및 상위 N개 선택
sorted_content_scores = sorted(content_similarity_scores, key=lambda x: x[1], reverse=True)
top_n_contents = sorted_content_scores[:10]

# 추천 사건명 출력
print("사용자 키워드와 관련된 사건명 :")
for 사건번호, score in top_n_contents:
    사건명 = df_inheritance.loc[df_inheritance['사건번호'] == 사건번호, '사건명'].values
    사건명 = 사건명[0] if 사건명.size > 0 else "사건명 없음"
    print(f"사건번호: {사건번호}, 사건명: {사건명}, 유사도: {score:.4f}")

사용자 키워드와 관련된 사건명 :
사건번호: 2006다46346, 사건명: 소유권이전등기등, 유사도: 0.3989
사건번호: 2010누30330, 사건명: 상속세 부과처분 취소, 유사도: 0.3938
사건번호: 81다카1061, 사건명: 부당이득반환, 유사도: 0.3917
사건번호: 2003다64381, 사건명: 종헌결의무효확인, 유사도: 0.3901
사건번호: 2017두44091, 사건명: 물납불허가처분취소청구, 유사도: 0.3869
사건번호: 2007카기134, 사건명: 위헌법률심판제청, 유사도: 0.3858
사건번호: 87누898, 사건명: 증여세등부과처분취소, 유사도: 0.3848
사건번호: 2021나31998, 사건명: 공유물분할, 유사도: 0.3843
사건번호: 91다27655, 사건명: 소유권이전등기, 유사도: 0.3841
사건번호: 68다2094, 사건명: 소유권이전등기말소등, 유사도: 0.3832


#### 판시사항

In [13]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 사용자 입력
user_keyword = "토지"  # 사용자가 입력한 키워드
user_keyword_embedding = encode_sentence(user_keyword)

# 판례내용 임베딩 유사도 계산 및 사건명 추출
content_similarity_scores = []

for idx, row in df_inheritance.iterrows():
    if row['판시사항임베딩'] is not None and not np.isnan(row['판시사항임베딩']).any():
        # 판례내용 임베딩과 사용자 키워드 임베딩 간의 유사도 계산
        content_similarity = cosine_similarity(user_keyword_embedding, row['판시사항임베딩']).flatten()[0]
        content_similarity_scores.append((row['사건번호'], content_similarity))

# 유사도 기준으로 정렬 및 상위 N개 선택
sorted_content_scores = sorted(content_similarity_scores, key=lambda x: x[1], reverse=True)
top_n_contents = sorted_content_scores[:10]

# 추천 사건명 출력
print("사용자 키워드와 관련된 사건명 :")
for 사건번호, score in top_n_contents:
    사건명 = df_inheritance.loc[df_inheritance['사건번호'] == 사건번호, '사건명'].values
    사건명 = 사건명[0] if 사건명.size > 0 else "사건명 없음"
    print(f"사건번호: {사건번호}, 사건명: {사건명}, 유사도: {score:.4f}")

사용자 키워드와 관련된 사건명 :
사건번호: 2007다9719, 사건명: 소유권이전등기, 유사도: 0.8622
사건번호: 83다카893, 사건명: 부당이득금, 유사도: 0.8622
사건번호: 83다카745, 사건명: 대여금, 유사도: 0.8622
사건번호: 81나459, 사건명: 공유물분할청구사건, 유사도: 0.8622
사건번호: 80다2360, 사건명: 소유권이전등기말소, 유사도: 0.8622
사건번호: 80나4240, 사건명: 손해배상청구사건, 유사도: 0.8622
사건번호: 79다434, 사건명: 소유권보존등기말소, 유사도: 0.8622
사건번호: 80다449, 사건명: 징발보상금, 유사도: 0.8622
사건번호: 79다1009, 사건명: 소유권보존등기말소, 유사도: 0.8622
사건번호: 79나1549, 사건명: 토지인도청구사건, 유사도: 0.8622
